# Primes Review

From [GeeksforGeeks Intro to Primality Tests](https://www.geeksforgeeks.org/introduction-to-primality-test-and-school-method/?ref=lbp)

## School Method:

Given a number _n_, iterated though all numbers from 2 to _n-1_ and for every number check if it divides _n_.  If it does, return false

**Time complexity:** O(n)
**Auxiliary Space:** O(1)

## Optimized School Method:

Instead of checking up to n, check up to $\sqrt n$, because any number higher than that must be a multiple of a smaller factor that has already been checked.  

**Time complexity:** O($\sqrt n$)
**Auxiliary Space:** O(1)

## Another Optimized School Method

All primes greater than three are of the form 6k $\pm$ 1, where _k_ is any integer greater than 0.  This works because all integers can be expressed as (6k + 1), where i = -1, 0, 1, 2, 3 or 4.  

So, first test whether _n_ is divisible by 2 or 3, and then check through all numbers of the form 6k $\pm$ 1 <= $\sqrt n$.  This is three times faster then checking all numbers up to $\sqrt n$.

In [4]:
import math

def isPrime(n):
  if n == 2 or n == 3:
    return True
  elif n <= 1 or n % 2 == 0 or n % 3 == 0:
    return False
  
  # Check through all numbers of the form 6k ± 1 
  # until i <= sqrt(n), with step value of 6
  for i in range(5, int(math.sqrt(n) + 1), 6):
    if n % i == 0 or n % (i+2) == 0:
      return False
      
  return True
  
# test it:
assert(isPrime(11) == True), 'Doesn\'t work!' 
assert(isPrime(20) == False), 'Doesn\'t work!' 

## Fermat Method of Primality Test

This is a probabilistic method:

If _n_ is a prime number, then for every _a_ where 1 < a < n-1

**a<sup>n-1</sup> % n = 1**

Example:

Since 5 is prime, 2<sup>4</sup> % 5 = 1

Equally, 3<sup>4</sup> % 5 = 1 and 4<sup>4</sup> % 5 = 1

Since 7 is prime, 2<sup>6</sup> % 7 = 1

Equally, 3<sup>6</sup> % 7 = 1, 4<sup>6</sup> % 7 = 1, 5<sup>6</sup> % 7 = 1, 6<sup>6</sup> % 7 = 1

If a given number is prime, this method always returns true.  If a given number is composite, this method may return true or false, but the probability of producing incorrect results is low and can be reduced by doing more iterations.  






Here is the algorithm:


A higher value of k indicates that the probability of correct results for composite inputs become higher. 

For prime inputs, the result is always correct.

1) Repeat k times:
   <ol type="a">
    <li>Pick <code>a</code> randomly in the range <code>[2, n-2]</code></li>
    <li>If <code>gcd(a, n) == 1</code>, return <code>false</code></li>
    <li>If a<sup>n-1</sup> != 1 (mod n) return <code>false</code></li>
    </ol>
2) Return `true`, the number  is probably prime

This code uses a modular exponentiation function called `power()`, which, given three numbers `x, y, p` computes (x<sup>y</sup>) % p 


In [6]:
# Iterative function to calculate (x^y) % p in O(log y) time

def power(x, y, p):
  # Initialized result
  res = 1
  
  while (y > 0):
    # if y is odd, multiply x with result
    if ((y & 1) != 0): # y is odd
      res = res * x

    # y must be even now
    y = y >> 1 # y = y / 2
    x = x * x # change x to x^2
  
  return res % p

x = 2
y = 5
p = 13
print("Power is ", power(x, y, p))

Power is  6


In [23]:
# Python 3 program to find the smallest
# twin in given range
import random

# power function mentioned just above
def power(a, n, p):
  # Initialized result
  res = 1
  
  # Update `a` if `a` >= p
  a = a % p
  
  while (n > 0):
    # if y is odd, multiply a with result
    if (n % 2): # n is odd
      res = (res * a) % p
      n = n - 1

    else:
      a = (a ** 2) % p
      
      # n must be even now
      n = n // 2
  
  return res % p

# If n is prime, then always returns true.
# If n is composite then returns false with 
#   high probability.
# A higher value of k increases probability of correct result
def ifPrime (n, k):
  # get the corner cases out of the way
  if n == 1 or n == 4:
    return False
  elif n == 2 or n == 3:
    return True
  
  # try k times
  else:
    for i in range(k):
      # Pick a random number in range [2..n-2]
      a = random.randint(2, n - 1)
      
      # Fermat's little theorem
      if power(a, n  - 1, n) != 1:
        return False
      
  # otherwise
  return True

# test it
def testIt(n, k):
  print(f"{n} is prime") if ifPrime(n, k) else print(f"{n} is not prime")
  
  
testIt(15, 3)
testIt(11, 3)
testIt(99, 3)
testIt(23, 3)
testIt(7841, 3)
testIt(7843, 3)
testIt(331999, 3)
# Carmichael number 561 = 3 * 11 * 17
testIt(561, 3) # this passes
testIt(561, 1) # this fails


15 is not prime
11 is prime
99 is not prime
23 is prime
7841 is prime
7843 is not prime
331999 is prime
561 is not prime
561 is prime


## Miller-Rabin Primality Test

Since Fermat can be fooled by [Carmicheal numbers](https://en.wikipedia.org/wiki/Carmichael_number) (see above example), the [Miller-Rabin](https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test) method is generally preferred.

**Algorithm:**
 
 It return `false` if n is composite and `true` if it is probably prime.  `k` is an input parameter that determines the accuracy level, where higher levels of k are more accurate.

 **`bool isPrime(int n, int k)`**

 1)  Handle base cases for `n < 3`
 2)  If `n` is even, return `false`
 3)  Find an odd number `d` such that `n-1` can be written as <code>d * 2<sup>r</sup></code>.  Since `n` is odd, `n - 1` must be even and `r` must be greater than 0.
 4)  Do the following `k` times:
   ```
   if (millerTest(n, d) == false)
      return false
   ```
 5)  Return `true`

This function is called for all `k` trials.  It returns `false` if `n` is composite and `true` if `n` is _probably_ prime.

`d` is an odd number such that <code>d*2<sup>r</sup></code> = `n-1` for some `r >= 1`

 **`bool millerTest(int n, int d)`**

 1) Pick a random number `a` in range `[2, n-2]`
 2) Compute: `x = pow(a, d) % n`
 3) If `x == 1` or `x == n - 1` return `true`
 4) While `d` doesn't become `n - 1`
    <ol type='a'>
    <li><code>x = (x * x) % n</code></li>
    <li><code>If (x == 1) return false</code></li>
    <li><code>If (x == n-1) return true</code></li>
    </ol>


In [31]:
# Python 3 Miller-Rabin primality test

# Utility function to do modular exponentiation.  
# See above for description and comments
def power(x, y, p):
  # initialize result:
  res = 1
  
  # Update x if it is >= p:
  x = x % p
  
  while (y > 0):
    if (y & 1): # y is odd
      res = (res * x) % p
      
    # y must be even
    y = y >> 1
    x = (x * x) % p

  return res

# This function is called for all k trials.  It returns
# False if n is composite and true if n is probably prime.
# d is an odd number such that d*2^r = n - 1 for some r >= 1
def millerTest(d, n):
  # Pick a random number in [2..n-2]
  # Corner cases make sure that n > 4
  a = 2 + random.randint(1, n - 4)
  
  # Compute a ^ d % n
  x = power(a, d, n)
  
  if (x == 1 or x == n - 1):
    return True; 
  
  # Keep squaring x while these things are true:
  # 1) d does not reach n - 1
  # 2) x ^ 2 % n is not 1
  # 3) x ^ % n is not n - 1
  while (d != n - 1):
    x = (x * x) % n
    d *= 2
    
    if (x == 1):
      return False
    if (x == n - 1):
      return True
    
  # n is composite number
  return False

# This function returns false if n is composite and true if
# n is _probably_ prime.  k is an input that determines the 
# accuracy level.  Higher k == higher accuracy.
def isPrime(n, k):
  # Corner cases
  if (n <= 1 or n == 4):
    return False
  if (n <= 3):
    return True
  
  # Find r such that n = 2 ^ d * r + 1 for some r > 1
  d = n - 1
  while (d % 2 == 0):
    d //= 2 # man, that's ugly
    
    # Iterate k times
    for i in range(k):
      if (millerTest(d, n) == False):
        return False
      
    return True
  
# Test it!
# iterations:
k = 4

# range
x = 100

print(f"All primes smaller than {x}:\n")
for n in range (1, x):
  if (isPrime(n, k)):
    print (n, end=" ")
  

# When k = 1 algo returns (21, 25 are composite!):
# 2 3 5 7 11 13 17 19 21 23 25 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 

# When k = 4 algo returns:
# 2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 


    

All primes smaller than 100:

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 